# <font color='red'>_Accidents</font> in Montreal: Data Cleaning_

- See the documentation [HERE](https://saaq.gouv.qc.ca/donnees-ouvertes/rapports-accident/rapports-accident-documentation.pdf)
- Get the data [HERE (http://donnees.ville.montreal.qc.ca/dataset/collisions-routieres)

#### Procedure
- Deleting Useless columns (see documentation)
- Make dummy variables for categorical/discrete variables

In [1]:
# imports
import pandas as pd
import numpy as np
import sys
import os 
os.chdir('c:/users/nicolas/documents/data/accidents-mtl')

In [2]:
# loading data
df = pd.read_csv('accidents_2012_2018.csv', index_col=0)

In [3]:
# resetting index
df = df.reset_index(drop=True)

In [4]:
# head
df.head()

,JR_SEMN_ACCDN,DT_ACCDN,CD_MUNCP,NO_CIVIQ_ACCDN,SFX_NO_CIVQ_ACCDN,BORNE_KM_ACCDN,RUE_ACCDN,TP_REPRR_ACCDN,ACCDN_PRES_DE,NB_METRE_DIST_ACCD,...,NB_VICTIMES_VELO,VITESSE_AUTOR,LOC_X,LOC_Y,LOC_COTE_Q,LOC_COTE_P,LOC_DETACHEE,LOC_IMPRECISION,LOC_LONG,LOC_LAT
0,ME,2012/02/01,66102.0,3501.0,NaN,NaN,ST CHARLES,2.0,STAT,NaN,...,0,NaN,276517.37950,5.035127e+06,A,3,O,N,-73.861616,45.455505
1,SA,2012/06/02,66023.0,NaN,NaN,NaN,NaN,NaN,COTE VERTU ET AUT 40,NaN,...,0,NaN,287913.26000,5.038666e+06,A,3,N,O,-73.716033,45.487715
2,JE,2012/06/28,66023.0,NaN,NaN,NaN,COTE VERTU,1.0,DECARIE,NaN,...,0,50.0,290518.82501,5.041617e+06,A,1,N,N,-73.682786,45.514324
3,ME,2012/07/11,66023.0,NaN,NaN,NaN,ST MATHIEU,1.0,RENE LEVESQUE,50.0,...,0,50.0,298822.88600,5.039146e+06,A,3,N,N,-73.576472,45.492212
4,LU,2012/01/02,66023.0,4849.0,NaN,NaN,ST JEAN,NaN,NaN,NaN,...,0,NaN,277433.35738,5.038881e+06,A,1,O,N,-73.850114,45.489319


#### Deleting Useless Columns

In [5]:
rows, columns = df.shape
print(f'We have {rows:,} rows and {columns} columns.')

We have 171,271 rows and 67 columns.


In [6]:
# deleting columns upon inspection of documentation
df.drop(['NO_CIVIQ_ACCDN', 'RUE_ACCDN', 'ACCDN_PRES_DE', 'CD_PNT_CDRNL_ROUTE', 
         'BORNE_KM_ACCDN', 'NB_METRE_DIST_ACCD', 'CD_PNT_CDRNL_REPRR', 
         'CD_SIT_PRTCE_ACCDN', 'nb_taxi', 'nb_urgence', 'nb_motoneige', 'nb_VHR', 
         'nb_autres_types', 'nb_veh_non_precise', 'CD_MUNCP', 'CD_ASPCT_ROUTE',
         'REG_ADM', 'MRC', 'LOC_DETACHEE', 'LOC_IMPRECISION', 'LOC_COTE_Q', 
         'LOC_COTE_P'], 
         axis=1, inplace=True)

In [7]:
rows, columns = df.shape
print(f'We have {rows:,} rows and {columns} columns.')

We have 171,271 rows and 47 columns.


In [8]:
# columns we are going to delete
null_col = df.loc[:, df.isnull().sum(axis=0) < 5e4].columns # del more than 50,000 missing values
null_num = df.loc[:, null_col].isnull().sum(axis=0)
null_count = pd.DataFrame([null_col, null_num]).T
null_count.columns = ['Name', 'Missing Values']
null_count.sort_values(by='Missing Values', ascending=False).head(10)

,Name,Missing Values
8,CD_CONFG_ROUTE,18738
7,CD_LOCLN_ACCDN,15009
9,CD_COND_METEO,11922
4,CD_ECLRM,11403
3,CD_ETAT_SURFC,11273
2,CD_GENRE_ACCDN,9045
5,CD_ENVRN_ACCDN,5957
6,CD_CATEG_ROUTE,5115
35,LOC_Y,11
34,LOC_X,11


In [9]:
# keeping only rows with less than 20% of missing values
df = df.loc[:, df.isnull().sum(axis=0) < 5e4]

In [10]:
# new shape
rows, columns = df.shape
print(f'We now have {rows:,} rows and {columns} columns.')

We now have 171,271 rows and 40 columns.


In [11]:
# new number of missing values 
missing_values = df.isnull().sum().sum()/df.size*100
print('We have {}% missing values.'.format(np.round(missing_values, 2)))

We have 1.29% missing values.


In [12]:
# size of the dataframe
print(f'Our dataframe is {int(sys.getsizeof(df)/1e6)} MB.')

Our dataframe is 112 MB.


In [13]:
# getting dtypes
column_types = df.dtypes
column_types.head()

JR_SEMN_ACCDN      object
DT_ACCDN           object
CD_GENRE_ACCDN    float64
CD_ETAT_SURFC     float64
CD_ECLRM          float64
dtype: object

In [14]:
# dropping rows missing our target
df.dropna(subset=['GRAVITE'], inplace=True)

In [15]:
# getting columns by datatype
integers = df.columns[column_types == 'int64']
integers

Index(['NB_MORTS', 'NB_BLESSES_GRAVES', 'NB_BLESS_LEGERS', 'AN',
       'NB_VICTIMES_TOTAL', 'NB_DECES_PIETON', 'NB_BLESSES_PIETON',
       'NB_VICTIMES_PIETON', 'NB_DECES_MOTO', 'NB_BLESSES_MOTO',
       'NB_VICTIMES_MOTO', 'NB_DECES_VELO', 'NB_BLESSES_VELO',
       'NB_VICTIMES_VELO', 'LOC_COTE_P'],
      dtype='object')

In [16]:
# getting columns by datatype
floats = df.columns[column_types == 'float64']
floats

Index(['CD_GENRE_ACCDN', 'CD_ETAT_SURFC', 'CD_ECLRM', 'CD_ENVRN_ACCDN',
       'CD_CATEG_ROUTE', 'CD_LOCLN_ACCDN', 'CD_CONFG_ROUTE', 'CD_COND_METEO',
       'NB_VEH_IMPLIQUES_ACCDN', 'nb_automobile_camion_leger',
       'nb_camionLourd_tractRoutier', 'nb_outil_equipement',
       'nb_tous_autobus_minibus', 'nb_bicyclette', 'nb_cyclomoteur',
       'nb_motocyclette', 'LOC_X', 'LOC_Y', 'LOC_LONG', 'LOC_LAT'],
      dtype='object')

In [17]:
# getting columns by datatype
objects = df.columns[column_types == 'object']
objects

Index(['JR_SEMN_ACCDN', 'DT_ACCDN', 'HR_ACCDN', 'GRAVITE', 'LOC_COTE_Q'], dtype='object')

In [19]:
# function to make dummies
def make_dummies(col):
    global df
    dummies = pd.get_dummies(df[col], prefix_sep=': ', prefix=col)
    df = pd.concat([df, dummies], sort=False, axis=1)
    df.drop(col, axis=1, inplace=True)

In [20]:
# function to see value counts
def vc(col):
    return df[col].value_counts()

In [21]:
# jour semaine
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('DI', 'Dimanche')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('LU', 'Lundi')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('MA', 'Mardi')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('ME', 'Mercredi')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('JE', 'Jeudi')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('VE', 'Vendredi')
df['JR_SEMN_ACCDN'] = df['JR_SEMN_ACCDN'].str.replace('SA', 'Samedi')

df['JR_SEMN_ACCDN'].value_counts()
make_dummies('JR_SEMN_ACCDN')

In [22]:
# month
month_dict = {
    '01':'Janvier',
    '02':'Février',
    '03':'Mars',
    '04':'Avril',
    '05':'Mai',
    '06':'Juin',
    '07':'Juillet',
    '08':'Août',
    '09':'Septembre',
    '10':'Octobre',
    '11':'Novembre',
    '12':'Décembre'
}
df['DT_ACCDN'] = df['DT_ACCDN'].str.split('/').str.get(1)
df['DT_ACCDN'] = df['DT_ACCDN'].astype(str).replace(month_dict)
make_dummies('DT_ACCDN')

In [23]:
# genre d'accident
genre_dict = {
    '31':'Collision avec véhicule routier',
    '32':'Collision avec piéton',
    '33':'Collision avec cycliste',
    '34':'Collision avec train',
    '35':'Collision avec chevreuil (cerf de Virginie)',
    '36':'Collision avec orignal/ours/caribou',
    '37':'Collision avec autre animal',
    '38':'Collision avec obstacle temporaire',
    '39':'Collision avec objet projeté/détaché',
    '40':'Objet fixe: lampadaire',
    '41':'Objet fixe: support/feu de signalisation',
    '42':'Objet fixe: poteau (service public)',
    '43':'Objet fixe: arbre',
    '44':'Objet fixe: section de glissière de sécurité',
    '45':'Objet fixe: atténuateur d’impact',
    '46':'Objet fixe: extrémité de glissière de sécurité',
    '47':'Objet fixe: pilier (pont/tunnel)',
    '48':'Objet fixe: amoncellement de neige',
    '49':'Objet fixe: bâtiment/édifice/mur',
    '50':'Objet fixe: bordure/trottoir',
    '51':'Objet fixe: borne-fontaine',
    '52':'Objet fixe: clôture/barrière',
    '53':'Objet fixe: fossé',
    '54':'Objet fixe: paroi rocheuse',
    '55':'Objet fixe: ponceau',
    '59':'Objet fixe: autre objet fixe',
    '71':'Sans collision: capotage',
    '72':'Sans collision: renversement',
    '73':'Sans collision: submersion/cours d’eau',
    '74':'Sans collision: feu/explosion',
    '75':'Sans collision: quitte la chaussée',
    '99':'Sans collision: autre'    
}
df['CD_GENRE_ACCDN'] = df['CD_GENRE_ACCDN'].astype(str).replace(genre_dict)
make_dummies('CD_GENRE_ACCDN')

In [25]:
# etat de la surface
surface_dict = {
    '11.0':'Sèche',
    '12.0':'Mouillée',
    '13.0':'Accumulation d\'eau',
    '14.0':'Sable, gravier',
    '15.0':'Gadoue',
    '16.0':'Enneigée',
    '17.0':'Neige durcie',
    '18.0':'Glacée',
    '19.0':'Boueuse',
    '20.0':'Huileuse',
    '90.0':'Autre'
}
df['CD_ETAT_SURFC'] = df['CD_ETAT_SURFC'].astype(str).replace(surface_dict)
make_dummies('CD_ETAT_SURFC')

In [26]:
# eclairement
ecl_dict = {
    '1':'Jour et clarté',
    '2':'Jour et demi-obscurité',
    '3':'Nuit et chemin éclairé',
    '4':'Nuit et chemin non éclairé'
}
make_dummies('CD_ECLRM')

In [27]:
# environment
env_dict = {
    '1':'Scolaire',
    '2':'Résidentiel',
    '3':'Commercial',
    '4':'Industriel',
    '5':'Rural',
    '6':'Forestier',
    '7':'Récréatif',
    '9':'Autre'
}
df['CD_ENVRN_ACCDN'] = df['CD_ENVRN_ACCDN'].astype(str).replace(env_dict)
make_dummies('CD_ENVRN_ACCDN')

In [28]:
# type route
categ_dict = {
    '11':'Chemin public: bretelle/collecteur d’autoroute/voie de service',
    '12':'Chemin public: route numérotée',
    '13':'Chemin public: artère principale',
    '14':'Chemin public: rue résidentielle',
    '15':'Chemin public: chemin/rang',
    '16':'Chemin public: ruelle',
    '19':'Chemin public: autre',
    '21':'Hors chemin public: terrain de stationnement',
    '22':'Hors chemin public: terrain privé',
    '23':'Hors chemin public: chemin privé',
    '24':'Hors chemin public: chemin forestier',
    '25':'Hors chemin public: sentier balisé',
    '29':'Hors chemin public: autre',
}
df['CD_CATEG_ROUTE'] = df['CD_CATEG_ROUTE'].astype(str).replace(categ_dict)
make_dummies('CD_CATEG_ROUTE')

In [29]:
# localisation
loc_dict = {
    '31':'Carrefour giratoire/rond-point',
    '32':'En intersection (moins de 5 mètres)',
    '33':'Près d’une intersection/carrefour giratoire',
    '34':'Entre intersections (100 mètres et +)',
    '35':'Passage à niveau',
    '36':'Pont (au-dessus d’un cours d’eau)',
    '37':'Autre pont (viaduc)',
    '38':'Tunnel',
    '39':'Sous un pont ou un viaduc',
    '40':'Centre commercial',
    '99':'Autres'
}
df['CD_LOCLN_ACCDN'] = df['CD_LOCLN_ACCDN'].astype(str).replace(loc_dict)
make_dummies('CD_LOCLN_ACCDN')

In [30]:
# configuration
config_dict = {
    '1':'Sens unique',
    '2':'Deux sens, une voie par direction',
    '3':'Deux sens, plus d’une voie par direction',
    '4':'Séparée par aménagement franchissable',
    '5':'Séparée par aménagement infranchissable',
    '6':'Autre (ex.: balises, voie de virage à gauche dans les deux sens)'
}
df['CD_CONFG_ROUTE'] = df['CD_CONFG_ROUTE'].astype(str).replace(config_dict)
make_dummies('CD_CONFG_ROUTE')

In [31]:
# meteo
meteo_dict = {
    '11':'Clair',
    '12':'Couvert',
    '13':'Brume',
    '14':'Pluie',
    '15':'Pluie forte',
    '16':'Vent fort',
    '17':'Neige',
    '18':'Poudrerie avec vent',
    '19':'Verglas',
    '10':'Autre'
}
df['CD_COND_METEO'] = df['CD_COND_METEO'].astype(str).replace(meteo_dict)
make_dummies('CD_COND_METEO')

In [32]:
# heure
heure_dict = {
    'Non précisé':'Inconnu',         
    '16:00:00-16:59:00':'16h__',
    '15:00:00-15:59:00':'15h__',
    '17:00:00-17:59:00':'17h__',
    '14:00:00-14:59:00':'14h__',
    '12:00:00-12:59:00':'12h__',
    '13:00:00-13:59:00':'13h__',
    '08:00:00-08:59:00':'08h__',
    '18:00:00-18:59:00':'18h__',
    '11:00:00-11:59:00':'11h__',
    '10:00:00-10:59:00':'10h__',
    '09:00:00-09:59:00':'09h__',
    '19:00:00-19:59:00':'19h__',
    '07:00:00-07:59:00':'07h__',
    '20:00:00-20:59:00':'20h__',
    '21:00:00-21:59:00':'21h__',
    '22:00:00-22:59:00':'22h__',
    '23:00:00-23:59:00':'23h__',
    '00:00:00-00:59:00':'00h__',
    '06:00:00-06:59:00':'06h__',
    '03:00:00-03:59:00':'03h__',
    '01:00:00-01:59:00':'01h__',
    '02:00:00-02:59:00':'02h__',
    '04:00:00-04:59:00':'04h__',
    '05:00:00-05:59:00':'05h__'
}
df['HR_ACCDN'] = df['HR_ACCDN'].astype(str).replace(heure_dict)
make_dummies('HR_ACCDN')

In [33]:
# gravite
df['GRAVITE'] = df['GRAVITE'].str.replace('inférieurs au seuil de rapportage', '(inférieurs)')
make_dummies('GRAVITE')

In [38]:
# final columns
df.columns[:28]

Index(['NB_VEH_IMPLIQUES_ACCDN', 'NB_MORTS', 'NB_BLESSES_GRAVES',
       'NB_BLESS_LEGERS', 'AN', 'NB_VICTIMES_TOTAL',
       'nb_automobile_camion_leger', 'nb_camionLourd_tractRoutier',
       'nb_outil_equipement', 'nb_tous_autobus_minibus', 'nb_bicyclette',
       'nb_cyclomoteur', 'nb_motocyclette', 'NB_DECES_PIETON',
       'NB_BLESSES_PIETON', 'NB_VICTIMES_PIETON', 'NB_DECES_MOTO',
       'NB_BLESSES_MOTO', 'NB_VICTIMES_MOTO', 'NB_DECES_VELO',
       'NB_BLESSES_VELO', 'NB_VICTIMES_VELO', 'LOC_X', 'LOC_Y', 'LOC_COTE_Q',
       'LOC_COTE_P', 'LOC_LONG', 'LOC_LAT'],
      dtype='object')

In [39]:
# final shape
rows, columns = df.shape
print(f'We have {rows:,} rows and {columns} columns.')

We have 171,271 rows and 179 columns.


In [40]:
# exports processed data to csv 
df.to_csv('a_dummies.csv', header=True, index=None)